In [1]:
!pip install pyyaml h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.11.0


# Load data

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
train_images.shape, test_images.shape

((1000, 784), (1000, 784))

# Create model

In [5]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


# Save model

## Save weights

In [6]:
checkpoint_path = os.path.join('training_1/cp.ckpt')
checkpoint_dir = os.path.dirname(checkpoint_path)

In [7]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [8]:
model.fit(
    train_images,
    train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    callbacks=[cp_callback]
)

Epoch 1/10
27/32 [========================>.....] - ETA: 0s - loss: 1.2319 - sparse_categorical_accuracy: 0.6481
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 15ms/step - loss: 1.1565 - sparse_categorical_accuracy: 0.6680 - val_loss: 0.6938 - val_sparse_categorical_accuracy: 0.7960
Epoch 2/10
28/32 [=========================>....] - ETA: 0s - loss: 0.4170 - sparse_categorical_accuracy: 0.8873
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 13ms/step - loss: 0.4212 - sparse_categorical_accuracy: 0.8830 - val_loss: 0.5298 - val_sparse_categorical_accuracy: 0.8410
Epoch 3/10
29/32 [==========================>...] - ETA: 0s - loss: 0.2856 - sparse_categorical_accuracy: 0.9256
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 12ms/step - loss: 0.2866 - sparse_categorical_accuracy: 0.9250 - val_loss: 0.4772 - val_sparse_categorical_accuracy: 0.8520
Epoch 4/10
28/32 [=======

In [9]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [10]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 [==============================] - 0s 3ms/step - loss: 2.4599 - sparse_categorical_accuracy: 0.0340
Untrained model, accuracy:  3.40%


In [11]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4280 - sparse_categorical_accuracy: 0.8610 - 133ms/epoch - 4ms/step
Restored model, accuracy: 86.10%


In [12]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = os.path.join("training_2/cp-{epoch:04d}.ckpt")
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [13]:
os.listdir(checkpoint_dir)

['cp-0025.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'checkpoint',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0050.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index']

In [14]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [15]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)

print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4856 - sparse_categorical_accuracy: 0.8750 - 163ms/epoch - 5ms/step
Restored model, accuracy: 87.50%


In [16]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4856 - sparse_categorical_accuracy: 0.8750 - 148ms/epoch - 5ms/step
Restored model, accuracy: 87.50%


## Save all model

In [17]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
32/32 [==============================] - 1s 6ms/step - loss: 1.1591 - sparse_categorical_accuracy: 0.6800
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.4192 - sparse_categorical_accuracy: 0.8860
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2985 - sparse_categorical_accuracy: 0.9260
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2185 - sparse_categorical_accuracy: 0.9380
Epoch 5/5
32/32 [==============================] - 0s 8ms/step - loss: 0.1470 - sparse_categorical_accuracy: 0.9700


In [18]:
!mkdir -p saved_model

In [19]:
model.save('saved_model/my_model')

In [21]:

!ls saved_model

!ls saved_model/my_model

my_model
assets	fingerprint.pb	keras_metadata.pb  saved_model.pb  variables


In [22]:
new_model = tf.keras.models.load_model('saved_model/my_model')
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4556 - sparse_categorical_accuracy: 0.8560 - 382ms/epoch - 12ms/step
Restored model, accuracy: 85.60%
32/32 [==============================] - 0s 2ms/step
(1000, 10)


In [24]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.1390 - sparse_categorical_accuracy: 0.6630
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.4064 - sparse_categorical_accuracy: 0.8890
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2802 - sparse_categorical_accuracy: 0.9310
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1936 - sparse_categorical_accuracy: 0.9540
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1553 - sparse_categorical_accuracy: 0.9700


In [25]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [26]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4380 - sparse_categorical_accuracy: 0.8550 - 252ms/epoch - 8ms/step
Restored model, accuracy: 85.50%
